In [14]:
import nltk
import pandas as pd

In [15]:
#load data
dataPath = "../data/"

fakeNewsCorpus = pd.read_csv(dataPath + "995,000_rows.csv")

/tmp/ipykernel_455730/146618987.py:4: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  fakeNewsCorpus = pd.read_csv(dataPath + "995,000_rows.csv")


# Exploring Data

In [16]:
#Hva saten er den der unnamed???
print(fakeNewsCorpus.head())

  Unnamed: 0         id               domain        type  \
0        732  7444726.0   nationalreview.com   political   
1       1348  6213642.0    beforeitsnews.com        fake   
2       7119  3867639.0     dailycurrant.com      satire   
3       1518  9560791.0          nytimes.com    reliable   
4       9345  2059625.0  infiniteunknown.net  conspiracy   

                                                 url  \
0  http://www.nationalreview.com/node/152734/%E2%...   
1  http://beforeitsnews.com/economy/2012/06/the-c...   
2  http://dailycurrant.com/2016/01/18/man-awoken-...   
3  https://query.nytimes.com/gst/fullpage.html?re...   
4  http://www.infiniteunknown.net/2011/09/14/100-...   

                                             content  \
0  Plus one article on Google Plus\n\n(Thanks to ...   
1  The Cost Of The Best Senate Banking Committee ...   
2  Man Awoken From 27-Year Coma Commits Suicide A...   
3  WHEN Julia Geist was asked to draw a picture o...   
4  – 100 Compiled Stud

In [17]:
print(fakeNewsCorpus.info())   # Check column types and missing values


fndf = pd.DataFrame(fakeNewsCorpus)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995000 entries, 0 to 994999
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        994999 non-null  object 
 1   id                994993 non-null  object 
 2   domain            994989 non-null  object 
 3   type              947214 non-null  object 
 4   url               994989 non-null  object 
 5   content           994988 non-null  object 
 6   scraped_at        994987 non-null  object 
 7   inserted_at       994987 non-null  object 
 8   updated_at        994987 non-null  object 
 9   title             986394 non-null  object 
 10  authors           552243 non-null  object 
 11  keywords          0 non-null       float64
 12  meta_keywords     956210 non-null  object 
 13  meta_description  469894 non-null  object 
 14  tags              230919 non-null  object 
 15  summary           0 non-null       float64
 16  source            21

In [18]:
print("Pandas DataFrame:")
display(fndf)


Pandas DataFrame:


,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,732,7444726.0,nationalreview.com,political,http://www.nationalreview.com/node/152734/%E2%...,Plus one article on Google Plus\n\n(Thanks to ...,2017-11-27T01:14:42.983556,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Iran News Round Up,NaN,NaN,"['National Review', 'National Review Online', ...",NaN,NaN,NaN,NaN
1,1348,6213642.0,beforeitsnews.com,fake,http://beforeitsnews.com/economy/2012/06/the-c...,The Cost Of The Best Senate Banking Committee ...,2017-11-27T01:14:08.7454,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,The Cost Of The Best Senate Banking Committee ...,NaN,NaN,[''],NaN,NaN,NaN,NaN
2,7119,3867639.0,dailycurrant.com,satire,http://dailycurrant.com/2016/01/18/man-awoken-...,Man Awoken From 27-Year Coma Commits Suicide A...,2017-11-27T01:14:21.395055,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Man Awoken From 27-Year Coma Commits Suicide A...,NaN,NaN,[''],NaN,NaN,NaN,NaN
3,1518,9560791.0,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,WHEN Julia Geist was asked to draw a picture o...,2018-02-11 00:46:42.632962,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,Opening a Gateway for Girls to Enter the Compu...,NaN,NaN,"['Computers and the Internet', 'Women and Girl...",WHEN Julia Geist was asked to draw a picture o...,NaN,NaN,nytimes
4,9345,2059625.0,infiniteunknown.net,conspiracy,http://www.infiniteunknown.net/2011/09/14/100-...,– 100 Compiled Studies on Vaccine Dangers (Act...,2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,100 Compiled Studies on Vaccine Dangers – Infi...,NaN,NaN,[''],NaN,"Lymphoma, Hepatitis B, Immune System, Health, ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994995,9376,1304673,21stcenturywire.com,conspiracy,http://21stcenturywire.com/tag/kony-2012/,By\n\n21WIRE + Intervention Watch | All we can...,2017-11-09T12:48:34.461199,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,KONY 2012 Archives,NaN,NaN,[''],NaN,"Michelle Obama, Drones, Selfie, Uganda, smart ...",NaN,NaN
994996,9625,7273484,nationalreview.com,political,http://www.nationalreview.com/node/152734/%E2%...,Plus one article on Google Plus\n\n(Thanks to ...,2017-11-27T01:14:42.983556,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Iran News Round Up,NaN,NaN,"['National Review', 'National Review Online', ...",NaN,NaN,NaN,NaN
994997,9561,3489380,thesaker.is,unknown,http://thesaker.is/china-and-russia-are-acquir...,"China and Russia are Acquiring Gold, Dumping U...",2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"China and Russia are Acquiring Gold, Dumping U...",The Saker,NaN,[''],A bird's eye view of the vineyard,"Gold, Dollar, Russia, China",NaN,NaN
994998,4293,6315455,express.co.uk,rumor,https://www.express.co.uk/pictures/sport/6708/...,Liverpool have signed Sadio Mane from Southamp...,2017-11-27T01:14:33.570665,2018-02-08 19:18:34.468038,2018-02-08 19:18:34.468066,Twitter reacts as Liverpool sign Sadio Mane,NaN,NaN,[''],Express Sport brings you all the best tweets a...,NaN,NaN,NaN


In [19]:
# Get the unique lable values
unique_values = fndf['type'].unique()
print(unique_values)

['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'unknown' nan 'clickbait' 'hate' 'junksci'
 '2018-02-10 13:43:39.521661']


In [20]:
# groups (reliable) as truenews 1 and (all others) in fakenews 0 
    #note this is naive and should be reconsidered later
fndf['type'] = fndf['type'].replace(r'^reliable$', '1', regex=True)  # Only replaces exact 'reliable' with 1
fndf['type'] = fndf['type'].replace(r'^(?!1$).+', '0', regex=True)   # Replace everything except '1' with '0'
#fndf['type'] = fndf['type'].fillna('0')

newunique_values = fndf['type'].unique()
print(newunique_values)
fndf.shape[0]


['0' '1' nan]


995000

In [ ]:
#hard to know how to classify nan, so removed for now
nan_count = fndf['type'].isna().sum()
print(f"Number of NaN instances: {nan_count}")
fndf = fndf.dropna(subset=['type'])
newunique_values = fndf['type'].unique()
print(newunique_values)


Number of NaN instances: 47786
['0' '1']


947214